In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
train_df=pd.read_csv('./yolodata/train.csv')
val_df=pd.read_csv('./yolodata/val.csv')

train_df_grouped = train_df.groupby('image_path')
val_df_grouped = val_df.groupby('image_path')

In [3]:
def clamp(n, minn, maxn):
    return max(min(maxn, n), minn)

In [4]:
import os

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov7(grouped_df,path):
    
    # For each bounding box
    for name,group in tqdm(grouped_df):

        bboxes=[]
        for i,row in group.iterrows():
          class_id = row["class"]
          b_center_x = (row["xmin"] + row["xmax"]) / 2 
          b_center_y = (row["ymin"] + row["ymax"]) / 2
          b_width    = (row["xmax"] - row["xmin"])
          b_height   = (row["ymax"] - row["ymin"])
          
          # Normalise the co-ordinates by the dimensions of the image
          # image_w, image_h, image_c = info_dict["image_size"]  
          b_center_x /= 960 
          b_center_y /= 540
          b_width    /= 960
          b_height   /= 540
          b_center_x = clamp(b_center_x,0,1)  
          b_center_y = clamp(b_center_y,0,1)
          b_width    = clamp(b_width,0,1) 
          b_height   = clamp(b_height,0,1)
          if class_id>=6:
            class_id-=1 ## not considering the class with only one image i.e class 6 BAD_STREETLIGHT
          bboxes.append("{:.0f} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        # Transform the bbox co-ordinates as per the format required by YOLO v8
        
        #Write the bbox details to the file 
        fname = os.path.join(path,name.replace("jpg","txt"))
        with open(fname,"w") as f:
          for box in bboxes:
            f.write(f"{box}\n")

In [5]:
convert_to_yolov7(train_df_grouped,'labels/train')

100%|██████████| 6299/6299 [16:26<00:00,  6.39it/s]


In [6]:
convert_to_yolov7(val_df_grouped,'labels/val')

100%|██████████| 1575/1575 [03:43<00:00,  7.03it/s]


Remove Image and its label for the BAD STREETLIGHT Class that contains only one image

In [ ]:
os.remove("/home/du4/19CS30043/hack/yolodata2/labels/train/53d3797457a0d2e3afe146e2f797e77e.txt")
os.remove("/home/du4/19CS30043/hack/yolodata2/labels/train/53d3797457a0d2e3afe146e2f797e77e.txt")